In [1]:
import urllib.request
import urllib.parse
import re
import bs4
import csv
import datetime
import string
import json
import os
from decimal import Decimal

In [15]:
def repl(text):
    return (text.replace('&#39;', ' ')
            .replace('\\n', '').replace('&quot;', ' ')).replace('\\', '').replace('&amp;', '&')


def filter_text(text):
        """Remove html tags"""
        tag_re = re.compile(r'<[^>]+>')
        printable = set(string.printable)
        wspace = re.compile(r'[\n\r\t]')
        xprob = re.compile(r'\\x[a-z0-9]{2}')
        return repl(' '.join(xprob.sub(' ', wspace.sub(' ', (''.join(filter(lambda x: x in printable, 
                                                                      tag_re.sub('', text))))
                                                 .replace('\\\'', '').replace('\"', ''))).split()))

In [3]:
class Project:
    
    def __init__(self, url):
        """Open url"""
        url = url.split('?')[0] + '/description'
        req = urllib.request.Request(url)
        resp = urllib.request.urlopen(req)
        respData = resp.read()
        self.url = url
        self.abstract = filter_text(re.findall(r'<meta property="og:description" content="(.*?)"/>',
                                               str(respData))[0])
        self.abstractLength = len(self.abstract)
        self.data = re.findall(r'<main role="main">(.*?)</main>', str(respData))[0]

        #print(re.findall(r'(usd|eur|gbp|aud|cad)', self.data))
        #if len(re.findall(r'<span class="money">(\$|€)', self.data)) == 0:
        #    raise Exception('Bad currency!')
        ctgry, subc = re.findall(r'mr3 nowrap type-12" href="/discover/categories/(.*?)/(.*?)\?ref', self.data)[0]
        self.category = ctgry.replace('%20', " ").replace('&amp;', "&")
        self.subcategory = subc.replace('%20', " ").replace('&amp;', "&")
        if len(self.subcategory) > 20:
            ctgry = re.findall(r'mr3 nowrap type-12" href="/discover/categories/(.*?)\?ref', self.data)[0]
            self.category = ctgry.replace('%20', " ").replace('&amp;', "&")
            self.subcategory = self.category
        # self.pcreator: 0 -> on fb, 1 -> num of of backed projects
        # 2 -> num of crested projects
        self.pcreator = [0, 0, 0]

            
    def find_features(self):
        """Project features"""
        sect = re.findall(r'<section class="NS(.*?)</section>', self.data)[0]
        self.title = filter_text(re.findall(r'<a .*>(.*?)</a>', re.findall(r'<section class="NS(.*?)</h2>', 
                                                                           self.data)[0])[0])
        self.titleLength = len(self.title)
        
        #creator features
        self.creator(re.findall(r'="About the creator" href="(.*?)"', sect)[0])
        
        #project features
        backers = re.findall(r'(\d+) backer', str(sect))
        if len(backers) == 0:
            self.backers = 0
            backers = re.findall(r'data-backers-count=\"(\d+)\"', str(sect))
            if len(backers) != 0:
                self.backers = int(float(backers[0]))
        else:
            self.backers = int(float(backers[0]))
        body = self.data[self.data.find(r'NS_projects__content'):]
        
        #self.updates = re.findall(r'<span class="count">(\d+)</span>', body)[0]
        #self.comments = re.findall(r'data-comments-count="(\d+)"', body)[0]
        #self.location = re.findall(r'discover/places/(.*?)\?ref=city', self.data)[0]

        GDB, EUR, AUD = 1.249, 1.064, 0.7445
        unit_goal = re.findall(r'(.{4})(\d+[\.|,]?\d+)</span> goal', body)
        if len(unit_goal) == 0:
            unit, goal = re.findall(r'(.{4})(\d+[\.|,]?\d+[\.|,]?\d+)</span> <span class="mobile-hide">goal', 
                                   sect)[0]
            self.goal = goal.replace(',','')
        else:
            unit, goal = unit_goal[0]
            self.goal = goal.replace(',','')
        self.pladged = re.findall(r'(\d+[\.|,]?\d+[\.|,]?\d+)</[a-z]{4}>', self.data)[0].replace(',','')
        
        if unit[0] == '2':
            g, p = float(str(float(self.goal))[1:]), float(str(float(self.pladged))[1:])
            self.goal = g * GDB
            self.pladged = p * GDB
        elif unit == "\\xac":
            self.goal = float(self.goal)*EUR
            self.pladged = float(self.pladged)*EUR
        elif unit[-1] == '$':
            self.goal = float(self.goal)
            self.pladged = float(self.pladged)
        elif unit == 'CA$ ' or unit == 'AU$ ':
            self.goal = float(self.goal)*AUD
            self.pladged = float(self.pladged)*AUD
        else:
            raise Exception('Bad currency!')
            
        self.goal, self.pladged = float(self.goal), float(self.pladged)
        success = self.pladged - self.goal
        if success >= 0:
            self.success = '1'
            if len(re.findall("Funding Unsuccessful", self.data)) >= 1:
                self.success = '0'
        else:
            self.success = '0'
            
        new = 0
        self.num_of_days = re.findall(r'(\d+) days\)', body)
        if len(self.num_of_days) == 0:      # for "first day" scraping
            self.success = '?'
            new = 1
        else:
            self.num_of_days = int(float(self.num_of_days[0]))
        
        if new:
            # for "first day" scraping
            date = re.findall(r'data-format="llll z" datetime="(\d+\-\d+\-\d+)T', self.data)[0]
            self.year = date.split("-")[0]
            month = date.split("-")[1]
            self.month = re.findall(r'data-format="llll z" datetime="(.*?)</time>', self.data)[0].split(",")[-1][1:4]
            date = datetime.date(int(self.year), int(month), int(date.split("-")[2]))
            now = datetime.date.today()
            self.num_of_days = (date - now).days
            self.datetime  = re.findall(r'data-format="llll z" datetime="(.*?)">', self.data)[0][:-6]
        else:
            self.month = re.findall(r'data-format="ll" datetime="(.*?)</time>', body)[0].split(" ")[-3][-3:]
            self.datetime = re.findall(r'data-format="ll" datetime="(.*?)">', body)[0][:-6]
            self.year = self.datetime[:4]
            
        rewards = re.findall(r'<li class="hover-group (.*?)</li>', body)
        if new:
            self.num_of_rewards = len(rewards) - 1
            self.min_reward = float(re.findall(r'About <span>\$(\d+\.?,?\d*)</span>', rewards[1])[0].replace(',',''))
            self.max_reward = float(re.findall(r'About <span>\$(\d+\.?,?\d*)</span>', rewards[-1])[0].replace(',','')
                                    .replace('\"',''))
        else:
            self.num_of_rewards = len(rewards)
            self.min_reward = float(re.findall(r'About <span>\$(\d+\.?,?\d*)</span>', rewards[0])[0].replace(',',''))
            self.max_reward = float(re.findall(r'About <span>\$(\d+\.?,?\d*)</span>', rewards[-1])[0].replace(',','')
                                    .replace('\"',''))
        
        #description features
        self.description(body)
        
        
        
    def creator(self, url):
        """Creator features"""
        req = urllib.request.Request('https://www.kickstarter.com' + url)
        resp = urllib.request.urlopen(req)
        respData = resp.read()
        
        connected = len(re.findall(r'Not connected', str(respData)))
        if connected == 0:
            self.pcreator[0] = 1
        else:
            self.pcreator[0] = 0
            
        n = re.findall(r'(\d+) backed', str(respData))
        if len(n) == 0:
            self.pcreator[1] = 0
        else:
            self.pcreator[1] = int(float(n[0]))
            
        n = re.findall(r'(\d+) created', str(respData))
        if len(n) == 0:
            self.pcreator[2] = 0
        else:
            self.pcreator[2] = int(float(n[0]))
            
        cdata = re.findall(r'<div class="readability">(.*?)</div>', str(respData))
        if len(cdata) > 0:
            self.cdataLength = len(cdata[0]) - 11
            self.cdata = filter_text(' '.join(cdata)).replace('\\n', '')
        else:
            self.cdataLength = 0 #!#ascii
            self.cdata = ""            
        
    
    def description(self, body):
        """Description features"""
        desc = re.findall(r'js-full-description responsive-media(.*?)project-faqs', body)[0]
        soup = bs4.BeautifulSoup(desc, "lxml")
        paragraphs = soup.findAll('p')
        self.num_characters = sum([len(str(p)) for p in paragraphs])
        paragraphs = [p.prettify(formatter=None) for p in paragraphs]
        self.text = filter_text(' '.join(paragraphs))[20:].replace('\\n', '')
        self.num_of_pictures = len(soup.findAll('figure'))
        soup2 = bs4.BeautifulSoup(self.data, "lxml")
        self.video = len(soup2.findAll('video'))
        self.hasVideo = 0  #!#
        if(int(self.video) > 0):
            self.hasVideo = 1
        self.faq = len(re.findall(r'<li class="faq"(.*?)>', body))

In [5]:
t = datetime.datetime.now()
print(t)
j = 0
k = 0

categories = [i for i in range(1, 363)] #24,362 # 17 - 165-200 manjka 28(40 naprej), manka 92 od 35(nism zihr) naprej

for c in categories:
    print("Category: ", c)
    with open('log.txt', 'a') as file:
        file.write("Category: " + str(c) + '\n')
    try:
        for i in range(100, 107):
            if i % 1 == 0:
                print("Page: ", i, ", Projects: ", k)
                with open('log.txt', 'a') as file:
                    file.write("Page: "+  str(i) + ", Projects: " + str(k) + '\n')
            url = 'https://www.kickstarter.com/discover/advanced?'
            values = {'category_id' : c, 'woe_id': '0', 'sort': 'end_date', 
                      'seed' : '2431954', 'page': i}
            data = urllib.parse.urlencode(values)
            reqUrl = url + str(data)
            req = urllib.request.Request(reqUrl)
            resp = urllib.request.urlopen(req)
            respData = resp.read()
            projects = re.findall(r'<li class="project col col-3 mb3 mb7-sm">(.*?)</li>', str(respData))
            for pr in projects:
                ahrefs = re.findall(r'href="(.*?)" target=""', str(pr))
                uns = re.findall(r'<div class="project-percent-pledged"(.*?)</div>', str(pr))
                if len(ahrefs) > 0 and len(uns) == 0:
                    try:
                        #if k % 4 == 0:
                        #    print(ahrefs[0])
                        wp = Project("https://www.kickstarter.com" + ahrefs[0])
                        wp.find_features()
                        jsn = {'Url': wp.url,
                               'Title' : wp.title,
                               'Campaign year': wp.year,
                               'Campaign month': wp.month,
                               'Category': wp.category, 
                               'Subcategory':  wp.subcategory,
                               'Author': 
                               {
                                    'Length of description': wp.cdataLength,
                                    'Description': wp.cdata, 
                                    'Facebook connection': wp.pcreator[0],
                                    'Number of backed projects': wp.pcreator[1], 
                                    'Number of created projects': wp.pcreator[2]
                                },
                               'Title length': wp.titleLength,
                               'Goal': wp.goal, 
                               'Duration': wp.num_of_days, 
                               'Number of pledge levels': wp.num_of_rewards, 
                               'Minimum pledge tiers': wp.min_reward,
                               'Maximum pledge tiers': wp.max_reward, 
                               'Length of project description': wp.num_characters, 
                               'Project description': wp.text,
                               'Abstract': wp.abstract,
                               'Length of abstract': wp.abstractLength,
                               'Number of images': wp.num_of_pictures, 
                               'Number of Faq items': wp.faq,
                               'Number of videos': wp.video, 
                               'Has a video': wp.hasVideo, 
                               'Success': wp.success
                              }
                        dirc = "data/" + str(wp.year) + "/" + str(wp.month)
                        if not os.path.exists(dirc):
                            os.makedirs(dirc)
                        with open(dirc + "/" + str(wp.datetime).replace(':', '').replace('-', '_') + '.json', 'w+') as fp:
                            json.dump(jsn, fp, indent=4)
                        k += 1
                    except Exception as e:
                        if i % 50 == 0:
                            print(e)
                        j += 1
                        continue
            if len(projects) < 10:
                break
    except Exception as e:
        print(e)
        continue
        
print("Errors: ", j)
print("Projects: ", k)
tK = datetime.datetime.now()
print("Time: ", abs(t-tK).seconds)

2017-01-06 11:47:55.228579
Category:  1
Page:  100 , Projects:  0
Page:  101 , Projects:  20
Page:  102 , Projects:  39


KeyboardInterrupt: 

In [20]:
import os
import json

def create_xlsx_file(start_year, end_year, fname):
    
    # define range 
    years = range(start_year, end_year)
    month = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]
    
    # define table of column's name - to ignore and not include in file 
    del_author_col = ['Description']
    del_project_col = ['Project description', 'Abstract']
    
    i = 0
    
    with open(fname + '.csv', 'w') as f:
        file = os.getcwd() + "/data/2016/Dec/2016_12_01T110010.json"
        writer = csv.writer(f)
        
        # read and filter header's elements from one project file
        header = []
        with open(file) as data_file:
            project = json.load(data_file)
            for el in del_author_col:
                del project["Author"][el]
            for el in del_project_col:
                del project[el]
            author_headers = list(project["Author"].keys())
            del project["Author"]
            header = author_headers + list(project.keys())
            
        # define discrete columns
        discrete_col = ['Category', 'Campaign month', 'Campaign year', 'Subcategory']
        
        # define string columns/meta data
        string_col = ['Url', 'Title']
        
        # create Three-Row Header Format for Orange analysis
        # defines attributes type (continuous - c, discrete - d and string - s)
        row02 = []
        # role (class, meta, weight, or ignore)
        row03 = []
        for h in header:
            if h == 'Success':
                row02 = row02 + ["c"]
                row03 = row03 + ["class"]
            elif h in discrete_col:
                row02 = row02 + ["d"]
                row03 = row03 + [""]
            elif h in string_col:
                row02 = row02 + ["s"]
                row03 = row03 + ["meta"]
            else:
                row02 = row02 + ["c"]
                row03 = row03 + [""]
           
        # write Three-Row Header Format in csv file
        writer.writerow(header)
        writer.writerow(row02)
        writer.writerow(row03)
        
        # read all *.json files created between start_year and end_year
        for y in years:
            for m in month:
                path = os.getcwd() + "/data/" + str(y) + "/" + m
                try:
                    for filename in os.listdir(path):
                        try:
                            project = {}
                            with open(path + "/" + filename) as data_file:

                                # create dict project
                                project = json.load(data_file)
                                i += 1
                                
                                # print progression
                                if i % 10000 == 0:
                                    print("Project: ", i)
                                    #with open('log.txt', 'a') as file:
                                    #   file.write("Project: "+  str(i) + '\n')
                                    
                                # give author attributes in project dict
                                for k in project["Author"].keys():
                                    project[k] = project["Author"][k]
                                    
                                # filter title
                                project["Title"] = filter_text(project["Title"])
                    
                                # sort attributes
                                project_row = [project[h] for h in header]
                            
                                # write all attribes in one row
                                writer.writerow(project_row)

                        except Exception as e:
                            print(e)
                            continue
                except Exception as e:
                    print(e)
                    continue
            
t = datetime.datetime.now()
print(t)

create_xlsx_file(2009, 2017, "kickstarter_data")

tK = datetime.datetime.now()
print("Time: ", abs(t-tK).seconds)

2017-01-06 16:33:46.916570
[Errno 2] No such file or directory: '/home/benjamin/Documents/kick/data/2009/Jan'
[Errno 2] No such file or directory: '/home/benjamin/Documents/kick/data/2009/Feb'
[Errno 2] No such file or directory: '/home/benjamin/Documents/kick/data/2009/Mar'
Project:  10000
Project:  20000
Project:  30000
Project:  40000
Project:  50000
Project:  60000
Project:  70000
Project:  80000
Project:  90000
Project:  100000
Project:  110000
Project:  120000
Project:  130000
Project:  140000
Project:  150000
Project:  160000
Project:  170000
Project:  180000
Time:  24


In [34]:
a = ['Number of created projects', 'Description', 'Length of description', 'Facebook connection', 
     'Number of backed projects', 'Category', 'Length of abstract', 'Number of Faq items', 
     'Title', 'Goal', 'Project description', 'Length of project description', 'Url', 'Has a video', 
     'Campaign month', 'Number of images', 'Abstract', 'Title length', 'Campaign year', 'Minimum pledge tiers', 
     'Number of videos', 'Duration', 'Success', 'Number of pledge levels', 'Subcategory', 'Maximum pledge tiers']

a = a.sort()
print(a)

['Number of created projects', 'Description', 'Length of description', 'Facebook connection', 'Number of backed projects', 'Category', 'Length of abstract', 'Number of Faq items', 'Title', 'Goal', 'Project description', 'Length of project description', 'Url', 'Has a video', 'Campaign month', 'Number of images', 'Abstract', 'Title length', 'Campaign year', 'Minimum pledge tiers', 'Number of videos', 'Duration', 'Success', 'Number of pledge levels', 'Subcategory', 'Maximum pledge tiers']
None


In [ ]:
t = datetime.datetime.now()
print(t)
j = 0
k = 0

categories = [i for i in range(34, 3620)] #24,362

for c in categories:
    b = False
    try:
        for i in range(1, 51):
            url = 'https://www.kickstarter.com/discover/advanced?'
            values = {'category_id' : c, 'woe_id': '0', 'sort': 'end_date', 
                      'seed' : '2431954', 'page': i}
            data = urllib.parse.urlencode(values)
            reqUrl = url + str(data)
            req = urllib.request.Request(reqUrl)
            resp = urllib.request.urlopen(req)
            respData = resp.read()
            projects = re.findall(r'<li class="project col col-3 mb3 mb7-sm">(.*?)</li>', str(respData))
            for pr in projects:
                ahrefs = re.findall(r'href="(.*?)" target=""', str(pr))
                uns = re.findall(r'<div class="project-percent-pledged"(.*?)</div>', str(pr))
                if len(ahrefs) > 0 and len(uns) == 0:
                    try:
                        wp = Project("https://www.kickstarter.com" + ahrefs[0])
                        wp.find_features()
                        dirc = "data/" + str(wp.year) + "/" + str(wp.month) + "/" + ...
                        str(wp.datetime).replace(':', '').replace('-', '_') + '.json'
                        if os.path.exists(dirc):
                            print("Category: ", c, ", obravnavano!")
                            b = True
                            break
                    except Exception as e:
                        j += 1
                        continue
            if len(projects) < 10:
                break
            if b:
                break
    except Exception as e:
        print(e)
        continue
    if not b:
        print("Category: ", c, ", NI!")
        
print("Errors: ", j)
print("Projects: ", k)
tK = datetime.datetime.now()
print("Time: ", abs(t-tK).seconds)